# Lista de exercícios 2

## Questão 1

In [1]:
import json
import numpy as np
from math import sin, cos, sqrt, atan2, radians

In [2]:
def distance_between_coords(coord1, coord2):
    """
    Function found in 
    https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
    """
    R = 6373.0
    lat1 = radians(coord1[1])
    lon1 = radians(coord1[0])
    lat2 = radians(coord2[1])
    lon2 = radians(coord2[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

In [18]:
with open ('coordinates.txt', 'r', encoding = 'UTF-8') as f:
    info = f.readlines()
lines = [txt.replace('\n', '') for txt in info if txt != '\n']
i = 0
coordinates = {}
while i < len(lines):
    key = lines[i].split('-')[0].strip()
    first_coord = lines[i+1].find('º')
    second_coord = lines[i+1].find('\'')
    third_coord = lines[i+1].find('"')
    lat = -1 * (int(lines[i+1][:first_coord]) + 
                int(lines[i+1][first_coord+1:second_coord])/60 + 
                int(lines[i+1][second_coord+1:third_coord])/3600)
    
    first_coord = lines[i+2].find('º')
    second_coord = lines[i+2].find('\'')
    third_coord = lines[i+2].find('"')
    
    lon = -1 * (int(lines[i+2][:first_coord]) + 
                int(lines[i+2][first_coord+1:second_coord])/60 + 
                int(lines[i+2][second_coord+1:third_coord])/3600)
    coordinates[key] = [lon, lat]
    i += 3

cities = list(coordinates.keys())
cities.remove('Macapá')
cities.remove('Palmas')
cities = cities[0:20]
cities_dict = dict([(cities[i], i) for i in range(len(cities))])

In [19]:
dists_euclid = np.zeros((len(cities), len(cities)))
for i in range(20):
    for j in range(20):
        dists_euclid[i, j] = distance_between_coords(coordinates[cities[i]], coordinates[cities[j]])

In [20]:
dists_road = np.zeros((len(cities), len(cities)))
with open("distancias.json", 'r', encoding='utf-8') as f:
    temp = json.load(f)
for i in range(20):
    for j in range(20):
        city1 = cities[i]
        city2 = cities[j]
        if city1+":"+city2 in temp.keys():
            dist = temp[city1+":"+city2]
        else:
            dist = temp[city2+":"+city1]
        dists_road[i, j] = dist

In [47]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, name = None, parent=None):
        self.parent = parent
        self.name = name
        self.g = 0
        self.h = 0
        self.f = 0
    
    def __eq__(self, other):
        return self.name == other.name

    
class A_star():
    def __init__(self, graph_dist, heuristic_dist, names):
        self.graph_dist = graph_dist
        self.heuristic_dist = heuristic_dist
        self.names = names
        self.names_dict = dict([(names[i], i) for i in range(len(names))])
        
    def run(self, start, end):
        print("A* algorithm - road between cities")
        print(f"start = {start}, end = {end}")
        
        openList = []
        closedList = []
        
        startNode = Node(start, None)
        startNode.g, startNode.h, startNode.f = 0, 0, 0
        
        openList.append(startNode)
        while len(openList) > 0:
            
            nodeBest = openList[0]
            valueBest = nodeBest.f
            for node in openList:
                if node.f < nodeBest.f:
                    nodeBest = node
            
            print(f"Current best = {nodeBest.name}, f = {nodeBest.f}")
            
            openList.remove(nodeBest)
            closedList.append(nodeBest)
            
            if nodeBest.name == end:
                print(f"Reached {end}")
                return 
            
            newPath = [Node(destination, nodeBest) for destination in self.names]
            for path in newPath:
                
                for closedPath in closedList:
                    if path == closedPath:
                        continue
                        
                path.g = nodeBest.g + self.graph_dist[self.names_dict[path.name],
                                                     self.names_dict[nodeBest.name]]
                
                path.h = self.heuristic_dist[self.names_dict[path.name],
                                            self.names_dict[end]]
                
                path.f = path.g + path.h
                
                for openPath in openList:
                    if path == openPath and path.g > openPath.g:
                        continue
                        
                print(f"Added new node for the search = {path.name}, g = {path.g}")
                openList.append(path)
                
            
            break
    
"""
def A_star(start, end):
    
    
    
    
    graph = dict([(city, {'f': np.inf, 'g': np.inf}) for city in cities])
    graph[start] = {'f': 0, 'g': 0}
    
    openList.append(start)
    
    while len(openList) > 0:
        nodeBest = ''
        currentBest = np.inf
        
        for node in openList:
            if graph[node]['f'] < currentBest:
                nodeBest = key
                currentBest = graph[node]['f']
        
        print(f"Current best = {node}, f({node}) = {graph[node]['f']}")
        if nodeBest == end:
            return
                            
        openList.remove(nodeBest)
        for city in cities:
            if city == start:
                continue
            elif:
                g = graph[nodeBest]['g'] + dist_road[cities_dict[nodeBest], cities_dict[city]]
                h = dist_euclid[cities_dict[city], cities_dict[end]]
                
                f = g + h
                
                if graph[city]['f'] < f:
                    continue
                else:
                    
                    
        
        closedList.append(nodeBest)
                    
        
        
        
            
        return



""";

In [48]:
A = A_star(dists_road, dists_euclid, cities)
A.run('Aracajú', 'Rio de Janeiro')

A* algorithm - road between cities
start = Aracajú, end = Rio de Janeiro
Current best = Aracajú, f = 0
Added new node for the search = Aracajú, g = 0.0
Added new node for the search = Belém, g = 2079.0
Added new node for the search = Belo Horizonte, g = 1578.0
Added new node for the search = Boa Vista, g = 6000.0
Added new node for the search = Brasília, g = 1652.0
Added new node for the search = Campo Grande, g = 2765.0
Added new node for the search = Cuiabá, g = 2775.0
Added new node for the search = Curitiba, g = 2595.0
Added new node for the search = Florianópolis, g = 2892.0
Added new node for the search = Fortaleza, g = 1183.0
Added new node for the search = Goiânia, g = 1848.0
Added new node for the search = João Pessoa, g = 611.0
Added new node for the search = Maceió, g = 294.0
Added new node for the search = Manaus, g = 5215.0
Added new node for the search = Natal, g = 788.0
Added new node for the search = Porto Alegre, g = 3296.0
Added new node for the search = Porto Velho, 